In [6]:
from influxdb import InfluxDBClient
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
import joblib


In [ ]:
influx_client = InfluxDBClient(host='localhost', port=8086, database='iot_s1')

query = 'SELECT * FROM "environment"'
result = influx_client.query(query)

points = list(result.get_points())
df = pd.DataFrame(points)

print(df.head)

In [ ]:
df.sort_values('time', inplace=True)

N = 3

features = []
targets = []

for i in range(N, len(df)):
    features.append(df[['temperature', 'humidity']].iloc[i-N:i].values)
    targets.append(df['temperature'].iloc[i])

features = np.array(features)
targets = np.array(targets)

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

scaler = MinMaxScaler(feature_range=(0, 1))
n_samples, n_timesteps, n_features = X_train.shape

X_train_flattened = X_train.reshape(n_samples * n_timesteps, n_features)
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_train_scaled = X_train_scaled.reshape(n_samples, n_timesteps, n_features)

X_test_flattened = X_test.reshape(X_test.shape[0] * n_timesteps, n_features)
X_test_scaled = scaler.transform(X_test_flattened)
X_test_scaled = X_test_scaled.reshape(X_test.shape[0], n_timesteps, n_features)

joblib.dump(scaler, 'scaler_sq.pkl')

model = Sequential([
    LSTM(200, return_sequences=True, input_shape=(n_timesteps, n_features)),
    Dropout(0.2),
    LSTM(200, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

optimizer = Adam(learning_rate=0.00001)

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_scaled, y_train, epochs=200, batch_size=16, validation_data=(X_test_scaled, y_test))

model.save('LSTM.h5')

In [ ]:
# Evaluation
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

model = load_model('LSTM.h5')

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)

rmse = np.sqrt(mse)

mae = mean_absolute_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absulote Error: {mae}')
print(f'R^2 Score: {r2}')

In [ ]:
# plotting the model
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(y_train, label='True Value')
plt.plot(y_pred, label='Predictions', linestyle='--')
plt.title('Comparison of actual and predicted temperatures')
plt.xlabel('Time Steps')
plt.ylabel('Temperature')
plt.legend()
plt.show()